In [1]:
import os
import argparse
import logging
logging.basicConfig(level=logging.ERROR)
from common import find_mxnet, data, fit
from common.util import download_file
import mxnet as mx
import numpy as np
import gzip, struct
from mxnet.symbol import *

In [2]:
parser = argparse.ArgumentParser(description="train mnist",
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('--num-classes', type=int, default=10,
                        help='the number of classes')
fit.add_fit_args(parser)
parser.set_defaults(
    # train
    gpus           = '5',
    batch_size     = 100,
    disp_batches   = 100,
    num_epochs     = 400,
    num_examples   = 100, 
    wd             = 1e-4,
    lr             = .001,
    lr_factor      = .33,
    optimizer      = 'adam',
    lr_step_epochs = '5000,10000,15000',
)
args = parser.parse_args("")
unsup_multiplier = 1
labeled_per_class = 10
sample_seed = 47
val_interval = 100
num_unsup_examples = 60000
num_sup_examples = labeled_per_class * args.num_classes
sup_batch_size = args.batch_size

In [3]:
def read_data(label, image):
    """
    download and read data into numpy
    """
    base_url = 'http://yann.lecun.com/exdb/mnist/'
    with gzip.open(download_file(base_url+label, os.path.join('data',label))) as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        label = np.fromstring(flbl.read(), dtype=np.int8)
    with gzip.open(download_file(base_url+image, os.path.join('data',image)), 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        image = np.fromstring(fimg.read(), dtype=np.uint8).reshape(len(label), rows, cols)
    return (label, image)


def to4d(img):
    """
    reshape to 4D arrays
    """
    return img.reshape(img.shape[0], 1, 28, 28).astype(np.float32)/255

def sample_by_label(images, labels, n_per_label, num_labels, seed=None):
    """Extract equal number of sampels per class."""
    res_img = []
    res_lbl = []
    rng = np.random.RandomState(seed=seed)
    for i in range(num_labels):
        a = images[labels == i]
        
        if n_per_label == -1:  # use all available labeled data
            res_img.append(a)
        else:  # use randomly chosen subset
            choice = rng.choice(len(a), n_per_label, False)
            print(choice)
            r = a[choice]
            
            res_img.append(r)
            
            lbls = np.ones(n_per_label) * i
            res_lbl.append(lbls)
    return (res_img, res_lbl)


"""
create data iterator with NDArrayIter
"""
(train_lbl, train_img) = read_data(
        'train-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte.gz')
(val_lbl, val_img) = read_data(
        't10k-labels-idx1-ubyte.gz', 't10k-images-idx3-ubyte.gz')

(imgs, lbls) = sample_by_label(to4d(train_img), train_lbl, labeled_per_class, 10, seed = sample_seed)
imgs = np.vstack(imgs)
lbls = np.hstack(lbls)

train_sup = mx.io.NDArrayIter(
    imgs, lbls, args.batch_size, shuffle=True, data_name='dataSup', label_name='labelSup')
train_unsup = mx.io.NDArrayIter(
    to4d(train_img), label=None, batch_size=args.batch_size, 
    shuffle=True, data_name='dataUnsup')
val = mx.io.NDArrayIter(
    to4d(val_img), val_lbl, args.batch_size*1) # use larger test batch size

[5840 5656  576 3662 3626 4448 4352 5659 4106 4176]
[1103 5635 3920 6070 6075 3763 6509 1527 1191 2650]
[5128 5073 2352 2550  164 5421 5858 2597 1559 3745]
[1750 1338 2470 1011 2191  774 4962 5773 6060 1810]
[2295 5622 5409  555 5314 2464  909 3597 1125 3279]
[5183 2728 2781 3977  345 4871 4113 1403 2388 1468]
[5142 1629  953 4917  950  672 2646 5433 3551 5325]
[5726 2479 1952 6120 4552 2228 2253 4124 4963 3627]
[4854  222  260 4818 1262 3203 5607  990 5167 1794]
[3534   41 4560 5218 4152  972 5041 2605  640 3111]


In [4]:
from common.multi_iterator import Multi_iterator
    
train = Multi_iterator(train_sup, train_unsup, unsup_multiplier, num_unsup_examples, num_sup_examples)

600  times more unsup data than sup data


In [5]:
from symbols import mnist
from common.lba import compute_semisup_loss, logit_loss
    
t_nb = sup_batch_size * unsup_multiplier

def build():
    dataSup = mx.symbol.Variable(name="dataSup")
    labelSup = mx.symbol.Variable(name='labelSup')
    overall_loss = []
    
    if unsup_multiplier >= 1:
        dataUnsup = []
        for i in range(unsup_multiplier):
            dataUnsup.append(Variable(name="dataUnsup"+str(i)))

        # concat data, feed both through the network
        # then split it up again
        data = concat(dataSup, *dataUnsup, dim=0)

        embeddings = mnist.build_embeddings(data)
        splitted = split(embeddings, num_outputs=(unsup_multiplier+1), axis=0, name='split')

        supEmbeddings = splitted[0]
        
        unsupEmbeddings = []
        for i in range(unsup_multiplier):
            unsupEmbeddings.append(splitted[1+i])
    
        unsupEmbeddings = concat(*unsupEmbeddings, dim=0)
            
        (walker_loss, visit_loss) = compute_semisup_loss(supEmbeddings, unsupEmbeddings, labelSup, t_nb, 
                                                     walker_weight=1.0, visit_weight=1.0)
        overall_loss = [walker_loss, visit_loss]
        
    else:
        supEmbeddings = mnist.build_embeddings(dataSup)
        
    overall_loss = [logit_loss(supEmbeddings, labelSup, 10)] + overall_loss
    
    return Group(overall_loss)

In [6]:
class Multi_Accuracy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self, num=None):
        super(Multi_Accuracy, self).__init__('multi-accuracy', num)

    def update(self, labels, preds):

        #for i in range(len(preds)):
        for i in range(1):
            pred_label = mx.nd.argmax_channel(preds[i]).asnumpy().astype('int32')
            label = labels[0].asnumpy().astype('int32')

            #mx.metric.check_label_shapes(label, pred_label)
            
            #print((pred_label.flat == label.flat).sum())
            #print(len(pred_label.flat))

            
            self.sum_metric[i] += (pred_label.flat == label.flat).sum()
            self.num_inst[i] += len(pred_label.flat)

In [7]:
# evaluation metrices
eval_metrics = Multi_Accuracy(num= 3 if unsup_multiplier >= 1 else 1)
                    
def fit_model(args, network, data, **kwargs):
    """
    train a model
    args : argparse returns
    network : the symbol definition of the nerual network
    data_loader : function that returns the train and val data iterators
    """
    # kvstore
    kv = mx.kvstore.create(args.kv_store)

    # logging
    head = '%(asctime)-15s Node[' + str(kv.rank) + '] %(message)s'
    logging.basicConfig(level=logging.ERROR, format=head)
    logging.info('start with arguments %s', args)
    batch_size = sup_batch_size * (unsup_multiplier + 1)
    

    # data iterators
    (train, val) = data
    if args.test_io:
        tic = time.time()
        for i, batch in enumerate(train):
            for j in batch.data:
                j.wait_to_read()
            if (i+1) % args.disp_batches == 0:
                logging.info('Batch [%d]\tSpeed: %.2f samples/sec' % (
                    i, args.disp_batches*batch_size/(time.time()-tic)))
                tic = time.time()
        return

    # load model
    if 'arg_params' in kwargs and 'aux_params' in kwargs:
        arg_params = kwargs['arg_params']
        aux_params = kwargs['aux_params']
    else:
        sym, arg_params, aux_params = fit._load_model(args, kv.rank)
        if sym is not None:
            assert sym.tojson() == network.tojson()

    # save model
    checkpoint = fit._save_model(args, kv.rank)

    # devices for training
    devs = mx.cpu() if args.gpus is None or args.gpus is '' else [
        mx.gpu(int(i)) for i in args.gpus.split(',')]

    # learning rate
    lr, lr_scheduler = fit._get_lr_scheduler(args, kv)

    data_names = ['dataSup'] + ['dataUnsup'+str(i) for i in range(unsup_multiplier)]
        
    # create model
    model = mx.mod.Module(
        context       = devs,
        symbol        = network,
        label_names   = ['labelSup'],
        data_names   = data_names
    )
    
    def validate_model(epoch, *args):
        if epoch % val_interval != 0: 
            return
        res = model.score(val, eval_metrics)
        #TODO: pull this into default
        for name, value in res:
            print('Epoch[%d] Validation-%s=%f' % (epoch, name, value))

    
    #print(model.label_names)
    lr_scheduler  = lr_scheduler
    optimizer_params = {
            'learning_rate': lr,
            #'momentum' : args.mom,
            'wd' : args.wd,
            'rescale_grad': 0.1,
            'lr_scheduler': lr_scheduler}

    #monitor = mx.mon.Monitor(interval=1000, pattern='.*aba_backward.*') 
    monitor = mx.mon.Monitor(interval=1000, pattern='.*') 
    
    #initializer = mx.init.Xavier(
    #    rnd_type='gaussian', factor_type="in", magnitude=2)
    initializer   = mx.init.Xavier(rnd_type='gaussian',factor_type="avg", magnitude=2.34)

    # callbacks that run after each batch
    batch_end_callbacks = [mx.callback.Speedometer(batch_size, args.disp_batches)]
    
    epoch_end_callbacks = validate_model

    # run
    model.fit(train,
        begin_epoch        = args.load_epoch if args.load_epoch else 0,
        num_epoch          = args.num_epochs,
        #eval_data          = val,
        eval_metric        = eval_metrics,
        kvstore            = kv,
        optimizer          = args.optimizer,
        optimizer_params   = optimizer_params,
        initializer        = initializer,
        arg_params         = arg_params,
        aux_params         = aux_params,
        #batch_end_callback = batch_end_callbacks,
        epoch_end_callback = epoch_end_callbacks,
        allow_missing      = True
        #monitor            = monitor
             )
    return model

In [9]:
train_sup.reset()
train_unsup.reset()

net = build()
import time
a = time.time()
model = fit_model(args, net, (train, val))
print(time.time()-a)

[5000.0, 10000.0, 15000.0]
Epoch[0] Validation-multi-accuracy_0=0.113200
Epoch[0] Validation-multi-accuracy_1=nan
Epoch[0] Validation-multi-accuracy_2=nan
Epoch[100] Validation-multi-accuracy_0=0.928600
Epoch[100] Validation-multi-accuracy_1=nan
Epoch[100] Validation-multi-accuracy_2=nan
Epoch[200] Validation-multi-accuracy_0=0.947400
Epoch[200] Validation-multi-accuracy_1=nan
Epoch[200] Validation-multi-accuracy_2=nan
Epoch[300] Validation-multi-accuracy_0=0.957200
Epoch[300] Validation-multi-accuracy_1=nan
Epoch[300] Validation-multi-accuracy_2=nan
19.018885612487793


In [ ]:
shape={"dataSup":(10, 1, 28,28),
                    "dataUnsup0":(10, 1, 28,28),
                    "dataUnsup1":(10, 1, 28,28), 
                     "labelSup": (10)
                    }

plot_network(net, shape=shape)

In [ ]:
batch=train.next()

labels = batch.label[0]
bDataSup = batch.data[0]
bDataUnsup = batch.data[1]


a = model.forward(batch)

In [ ]:
def confusion_matrix(labels, predictions, num_labels):
  """Compute the confusion matrix."""
  rows = []
  for i in range(num_labels):
    row = np.bincount(predictions[labels == i], minlength=num_labels)
    rows.append(row)
  return np.vstack(rows)

val.reset()
res = model.predict(val)[0].asnumpy().argmax(axis=1)
print(res.shape, val_lbl.shape)

confusion_matrix(val_lbl, res,10)

In [ ]:
print(labels.asnumpy())
model.get_outputs()[1].asnumpy()[0:10,0:10]